# Create function to analyze customers' shopping preference & advise on targeted products

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lifetimes

from datetime import datetime
from dateutil.relativedelta import relativedelta

import os
from google.cloud import bigquery
client = bigquery.Client()

### Import Data

In [5]:
# Import order data
gcr_project_id = os.getenv('GCR_CLV_PROJECT_ID')

QUERY  = f"""
SELECT
  order_items.user_id,
  users.first_name,
  users.last_name,
  order_items.order_id,
  order_items.product_id,
  products.name as product_name,
  products.brand as product_brand,
  order_items.sale_price,
  order_items.status
FROM `ecommerce-data-project-444616.thelook_ecommerce.order_items` as order_items
LEFT JOIN `ecommerce-data-project-444616.thelook_ecommerce.users` as users
ON order_items.user_id = users.id
LEFT JOIN `ecommerce-data-project-444616.thelook_ecommerce.products` as products
ON order_items.product_id = products.id
ORDER BY order_items.user_id;
"""

df_orders = client.query_and_wait(QUERY).to_dataframe()
df_orders.head()

/Users/axel/ecommerce-project/env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,user_id,first_name,last_name,order_id,product_id,product_name,product_brand,sale_price,status
0,1,James,Bell,1,23228,Lee Men's Big & Tall Belted Wyoming Cargo Short,Lee,39.880001,Processing
1,1,James,Bell,2,17564,TapouT Men's Top Contender Hoodie,TapouT,56.000000,Processing
2,1,James,Bell,3,19198,RVCA Men's Rugged Sweater,RVCA,60.000000,Complete
3,2,Sandra,Rogers,4,9167,HUE Women's Relaunch Tulle Dot,HUE,11.120000,Shipped
4,2,Sandra,Rogers,4,2104,Allegra K Woman Schoolbag Pattern Back Drawstr...,Allegra K,13.910000,Shipped


In [6]:
QUERY  = f"""
SELECT *
FROM `ecommerce-data-project-444616.thelook_ecommerce.products`;
"""

df_products = client.query_and_wait(QUERY).to_dataframe()
df_products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


### Create LLM for product similarity evaluation